# Working with Pandas and SQL Databases (Movies Dataset)

## Creating an SQLite Database

1. __Import__ sqlite3 (as sq3) and __create__ a new SQLite Database with the name __"movies.db"__.

In [1]:
import pandas as pd
import sqlite3 as sq3
import json

In [2]:
#to connect to db in working directory. If does not exist, will be created
con = sq3.connect("movies.db")

In [3]:
con

In [4]:
#to close db

con.close()

## Loading Data from DataFrames into an SQLite Database

2. __Load__ the json file __"some_movies.json"__ and __split__ the dataset into the following __four datasets__ (save each dataset in a Pandas DataFrame).

In [5]:
with open('some_movies.json') as f:
    data = json.load(f)

In [6]:
# import json data into df

df = pd.json_normalize(data, sep='_')
df.head(1)

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,belongs_to_collection_id,belongs_to_collection_name,belongs_to_collection_poster_path,belongs_to_collection_backdrop_path,belongs_to_collection
0,False,/orjiB3oUIsyz60hoEqkiGpy5CeO.jpg,356000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 878, ...",https://www.marvel.com/movies/avengers-endgame,299534,tt4154796,en,Avengers: Endgame,After the devastating events of Avengers: Infi...,...,Part of the journey is the end.,Avengers: Endgame,False,8.3,12607,86311.0,The Avengers Collection,/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg,/zuW6fOiusv4X9nnW3paHGfXcSll.jpg,NaN


__Dataset #1 (Movies)__ with columns ["id", "title", "revenue", "budget", "belongs_to_collection_name", "release_date"]. <br>
Convert "release_date" to datetime

In [7]:
movies = df[["id", "title", "revenue", "budget", "belongs_to_collection_name", "release_date"]].copy()
movies.head(1)

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,2797800564,356000000,The Avengers Collection,2019-04-24


In [8]:
movies['release_date'] = pd.to_datetime(movies['release_date'])

__Dataset #2 (Votes)__ with columns ["id", "vote_count", "vote_average"]. 

In [9]:
votes = df[["id", "vote_count", "vote_average"]].copy()
votes.head(1)

,id,vote_count,vote_average
0,299534,12607,8.3


__Dataset #3 (Genres)__ with columns ["genre_id", "genre_name", "id"]. <br> 

In [10]:
genres = pd.json_normalize(data, record_path='genres', meta='id', record_prefix='genre_')
genres.head()

,genre_id,genre_name,id
0,12,Adventure,299534
1,878,Science Fiction,299534
2,28,Action,299534
3,28,Action,19995
4,12,Adventure,19995


__Dataset #4 (Prod)__ with columns ["comp_id", "comp_logo_path", "comp_name", "comp_origin_country", "id" ]. <br>


In [11]:
prod = pd.json_normalize(data, record_path='production_companies', meta='id',record_prefix='comp_')
prod.head()

,comp_id,comp_logo_path,comp_name,comp_origin_country,id
0,420,/hUzeosd33nzE5MCNsZxCGEKTXaQ.png,Marvel Studios,US,299534
1,444,/42UPdZl6B2cFXgNUASR8hSt9mpS.png,Dune Entertainment,US,19995
2,574,/iB6GjNVHs5hOqcEYt2rcjBqIjki.png,Lightstorm Entertainment,US,19995
3,25,/qZCc1lty5FzX30aOCVRBLzaVmcp.png,20th Century Fox,US,19995
4,290,/Q8mw2AOQQc8Qg0uNwLWq86DVZv.png,Ingenious Media,GB,19995


3. __Load__ the datasets __into the database__ (each dataset should be a separate table in the database). 

## Loading Data from SQLite Databases into DataFrames

4. __Load__ the full tables "Movies", "Votes", "Genres", "Prod" from "movies.db" into Pandas (four DataFrames). __Set__ "id" as Index. 

In [12]:
# 1. create connection

con = sq3.connect('movies.db')
con

In [13]:
movies.to_sql('Movies', con, index=False)
votes.to_sql("Votes", con, index=False)
genres.to_sql("Genres", con, index=False)
prod.to_sql("Prod", con, index=False)

52

##  Some Simple SQL Queries

5. __Perform__ the following simple __SQL Queries__ 

__Load the full "Movies" Table__.

In [15]:
con.execute('select * from sqlite_master').fetchall()

[('table',
  'Movies',
  'Movies',
  2,
  'CREATE TABLE "Movies" (\n"id" INTEGER,\n  "title" TEXT,\n  "revenue" INTEGER,\n  "budget" INTEGER,\n  "belongs_to_collection_name" TEXT,\n  "release_date" TIMESTAMP\n)'),
 ('table',
  'Votes',
  'Votes',
  3,
  'CREATE TABLE "Votes" (\n"id" INTEGER,\n  "vote_count" INTEGER,\n  "vote_average" REAL\n)'),
 ('table',
  'Genres',
  'Genres',
  4,
  'CREATE TABLE "Genres" (\n"genre_id" INTEGER,\n  "genre_name" TEXT,\n  "id" INTEGER\n)'),
 ('table',
  'Prod',
  'Prod',
  5,
  'CREATE TABLE "Prod" (\n"comp_id" INTEGER,\n  "comp_logo_path" TEXT,\n  "comp_name" TEXT,\n  "comp_origin_country" TEXT,\n  "id" INTEGER\n)')]

In [16]:
pd.read_sql('select * from Movies', con, index_col='id', parse_dates='release_date')

,title,revenue,budget,belongs_to_collection_name,release_date
id,,,,,
299534,Avengers: Endgame,2797800564,356000000,The Avengers Collection,2019-04-24
19995,Avatar,2787965087,237000000,Avatar Collection,2009-12-10
140607,Star Wars: The Force Awakens,2068223624,245000000,Star Wars Collection,2015-12-15
299536,Avengers: Infinity War,2046239637,300000000,The Avengers Collection,2018-04-25
597,Titanic,1845034188,200000000,None,1997-11-18
135397,Jurassic World,1671713208,150000000,Jurassic Park Collection,2015-06-06
420818,The Lion King,1656943394,260000000,None,2019-07-12
24428,The Avengers,1519557910,220000000,The Avengers Collection,2012-04-25
168259,Furious 7,1515047671,190000000,The Fast and the Furious Collection,2015-04-01


__Load the columns "id", "revenue" and "release_date" from "Movies".__ 

In [17]:
pd.read_sql('''select id, revenue, release_date from Movies''', con)

,id,revenue,release_date
0,299534,2797800564,2019-04-24 00:00:00
1,19995,2787965087,2009-12-10 00:00:00
2,140607,2068223624,2015-12-15 00:00:00
3,299536,2046239637,2018-04-25 00:00:00
4,597,1845034188,1997-11-18 00:00:00
5,135397,1671713208,2015-06-06 00:00:00
6,420818,1656943394,2019-07-12 00:00:00
7,24428,1519557910,2012-04-25 00:00:00
8,168259,1515047671,2015-04-01 00:00:00
9,99861,1405403694,2015-04-22 00:00:00


__Get the Total Revenue (sum) over all movies from "Movies".__

In [18]:
pd.read_sql('''select sum(revenue) from Movies''', con)

,sum(revenue)
0,29748575327


__Count the number of Movies in "Movies".__

In [19]:
pd.read_sql('''select count(*) from Movies''', con)

,count(*)
0,18


__Count the number of Movies that do belong to a collection.__

In [20]:
pd.read_sql('''select count(*) from Movies where belongs_to_collection_name isnull ''', con)

,count(*)
0,3


__Get the average budget from "Movies"__.

In [21]:
pd.read_sql('''select avg(budget) from Movies''', con)

,avg(budget)
0,2.090556e+08


## Some more SQL Queries

6. __Perform__ the following advanced __SQL Queries__ and __store__ the results in DataFrames:

__Load all columns for the movie with movie id 597__.

In [22]:
pd.read_sql('''select * from Movies where id=597''', con)

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,597,Titanic,1845034188,200000000,None,1997-11-18 00:00:00


__Load all columns for all movies with a revenue greater than 2000 (MUSD).__

In [23]:
# function for repetative method

def my_sql(query):
    return pd.read_sql(query, con)

my_sql('''select * from Movies where revenue >2000000000''')

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,2797800564,356000000,The Avengers Collection,2019-04-24 00:00:00
1,19995,Avatar,2787965087,237000000,Avatar Collection,2009-12-10 00:00:00
2,140607,Star Wars: The Force Awakens,2068223624,245000000,Star Wars Collection,2015-12-15 00:00:00
3,299536,Avengers: Infinity War,2046239637,300000000,The Avengers Collection,2018-04-25 00:00:00


__Load all columns for all movies with a revenue greater than 1500 (MUSD) and a budget below 200 (MUSD).__

In [24]:
my_sql('select * from movies where revenue between 200000000 and 1500000000')

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,99861,Avengers: Age of Ultron,1405403694,250000000,The Avengers Collection,2015-04-22 00:00:00
1,284054,Black Panther,1346739107,200000000,Black Panther Collection,2018-02-13 00:00:00
2,12445,Harry Potter and the Deathly Hallows: Part 2,1341511219,125000000,Harry Potter Collection,2011-07-07 00:00:00
3,181808,Star Wars: The Last Jedi,1332539889,200000000,Star Wars Collection,2017-12-13 00:00:00
4,330457,Frozen II,1330764959,150000000,Frozen Collection,2019-11-20 00:00:00
5,351286,Jurassic World: Fallen Kingdom,1303459585,170000000,Jurassic Park Collection,2018-06-06 00:00:00
6,109445,Frozen,1274219009,150000000,Frozen Collection,2013-11-27 00:00:00
7,321612,Beauty and the Beast,1263521126,160000000,None,2017-03-16 00:00:00
8,260513,Incredibles 2,1241891456,200000000,The Incredibles Collection,2018-06-14 00:00:00


__Get the minimum budget from those movies with a revenue greater than 1250 (MUSD).__

In [25]:
my_sql('select min(budget) from movies where revenue > 1250000000')

,min(budget)
0,125000000


__Get all unique collection Names from "Movies".__

In [26]:
my_sql('select distinct(belongs_to_collection_name) from movies')

,belongs_to_collection_name
0,The Avengers Collection
1,Avatar Collection
2,Star Wars Collection
3,None
4,Jurassic Park Collection
5,The Fast and the Furious Collection
6,Black Panther Collection
7,Harry Potter Collection
8,Frozen Collection
9,The Incredibles Collection


__Load all movies (all columns) and sort by budget from high to low.__

In [27]:
my_sql('select * from movies order by budget desc')

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,2797800564,356000000,The Avengers Collection,2019-04-24 00:00:00
1,299536,Avengers: Infinity War,2046239637,300000000,The Avengers Collection,2018-04-25 00:00:00
2,420818,The Lion King,1656943394,260000000,None,2019-07-12 00:00:00
3,99861,Avengers: Age of Ultron,1405403694,250000000,The Avengers Collection,2015-04-22 00:00:00
4,140607,Star Wars: The Force Awakens,2068223624,245000000,Star Wars Collection,2015-12-15 00:00:00
5,19995,Avatar,2787965087,237000000,Avatar Collection,2009-12-10 00:00:00
6,24428,The Avengers,1519557910,220000000,The Avengers Collection,2012-04-25 00:00:00
7,597,Titanic,1845034188,200000000,None,1997-11-18 00:00:00
8,284054,Black Panther,1346739107,200000000,Black Panther Collection,2018-02-13 00:00:00
9,181808,Star Wars: The Last Jedi,1332539889,200000000,Star Wars Collection,2017-12-13 00:00:00


__Load all movies (all columns) that do not belong to a collection.__

In [28]:
my_sql('select * from movies where belongs_to_collection_name isnull')

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,597,Titanic,1845034188,200000000,None,1997-11-18 00:00:00
1,420818,The Lion King,1656943394,260000000,None,2019-07-12 00:00:00
2,321612,Beauty and the Beast,1263521126,160000000,None,2017-03-16 00:00:00


__Load all movies (all columns) that belong to a collection.__

In [29]:
my_sql('select * from movies where belongs_to_collection_name is not null')

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,2797800564,356000000,The Avengers Collection,2019-04-24 00:00:00
1,19995,Avatar,2787965087,237000000,Avatar Collection,2009-12-10 00:00:00
2,140607,Star Wars: The Force Awakens,2068223624,245000000,Star Wars Collection,2015-12-15 00:00:00
3,299536,Avengers: Infinity War,2046239637,300000000,The Avengers Collection,2018-04-25 00:00:00
4,135397,Jurassic World,1671713208,150000000,Jurassic Park Collection,2015-06-06 00:00:00
5,24428,The Avengers,1519557910,220000000,The Avengers Collection,2012-04-25 00:00:00
6,168259,Furious 7,1515047671,190000000,The Fast and the Furious Collection,2015-04-01 00:00:00
7,99861,Avengers: Age of Ultron,1405403694,250000000,The Avengers Collection,2015-04-22 00:00:00
8,284054,Black Panther,1346739107,200000000,Black Panther Collection,2018-02-13 00:00:00
9,12445,Harry Potter and the Deathly Hallows: Part 2,1341511219,125000000,Harry Potter Collection,2011-07-07 00:00:00


__Load all movies (all columns) where "Avengers..." is in the title__.

In [30]:
my_sql('select * from movies where title like "%Avengers%"')

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,2797800564,356000000,The Avengers Collection,2019-04-24 00:00:00
1,299536,Avengers: Infinity War,2046239637,300000000,The Avengers Collection,2018-04-25 00:00:00
2,24428,The Avengers,1519557910,220000000,The Avengers Collection,2012-04-25 00:00:00
3,99861,Avengers: Age of Ultron,1405403694,250000000,The Avengers Collection,2015-04-22 00:00:00


## Join Queries

7. __Perform__ the following __SQL Join Queries__ and __store__ the results in DataFrames:

__Join "Movies" and "Votes"__ (all columns).

In [31]:
pd.read_sql('select * from movies full join votes on votes.id=movies.id', con)

,id,title,revenue,budget,belongs_to_collection_name,release_date,id,vote_count,vote_average
0,299534,Avengers: Endgame,2797800564,356000000,The Avengers Collection,2019-04-24 00:00:00,299534,12607,8.3
1,19995,Avatar,2787965087,237000000,Avatar Collection,2009-12-10 00:00:00,19995,21000,7.4
2,140607,Star Wars: The Force Awakens,2068223624,245000000,Star Wars Collection,2015-12-15 00:00:00,140607,14205,7.4
3,299536,Avengers: Infinity War,2046239637,300000000,The Avengers Collection,2018-04-25 00:00:00,299536,17718,8.3
4,597,Titanic,1845034188,200000000,None,1997-11-18 00:00:00,597,16661,7.8
5,135397,Jurassic World,1671713208,150000000,Jurassic Park Collection,2015-06-06 00:00:00,135397,15399,6.6
6,420818,The Lion King,1656943394,260000000,None,2019-07-12 00:00:00,420818,5425,7.2
7,24428,The Avengers,1519557910,220000000,The Avengers Collection,2012-04-25 00:00:00,24428,22101,7.7
8,168259,Furious 7,1515047671,190000000,The Fast and the Furious Collection,2015-04-01 00:00:00,168259,7359,7.3
9,99861,Avengers: Age of Ultron,1405403694,250000000,The Avengers Collection,2015-04-22 00:00:00,99861,15548,7.3


__Join "Movies" and "Votes" (only the columns "id", "title", "vote_average").__

__Join "Movies" and "Votes" (only the columns "id", "title", "vote_average") and return only those movies with vote_average > 8.__

In [32]:
pd.read_sql('select m.id, m.title, v.vote_average from votes v join movies m on m.id=v.id where v.vote_average>8', con)

,id,title,vote_average
0,299534,Avengers: Endgame,8.3
1,299536,Avengers: Infinity War,8.3
2,12445,Harry Potter and the Deathly Hallows: Part 2,8.1


__Join "Movies" and "Votes" (only the columns "id", "title", "vote_average") and return only those movies with vote_average > 8 and in ascending budget order__.

## Final Case Study

8. __Perform__ the following advanced __SQL Queries__ and __store__ the results in DataFrames:

__Get the Total Revenue (sum) for each Production Company.__

In [33]:
my_sql('''select p.comp_name, sum(m.revenue) 
from movies m join prod p on p.id=m.id 
group by p.comp_name 
order by sum(m.revenue) desc''')

,comp_name,sum(m.revenue)
0,Walt Disney Pictures,9446618940
1,Marvel Studios,9115740912
2,Lightstorm Entertainment,4632999275
3,20th Century Fox,4632999275
4,Universal Pictures,4490220464
5,Lucasfilm,3400763513
6,Paramount,3364592098
7,Fuji Television Network,3186760879
8,Dentsu,3186760879
9,Legendary Entertainment,2975172793


__Get all Production Companies for the movie "Titanic".__

In [34]:
my_sql('''select distinct(p.comp_name)
from Prod p join movies m on m.id=p.id
where m.title = "Titanic"''')

,comp_name
0,20th Century Fox
1,Lightstorm Entertainment
2,Paramount


__Get the Total Revenue (sum) for each Genre.__

In [35]:
my_sql('''select g.genre_name, sum(m.revenue)
from movies m join genres g on g.id=m.id
group by g.genre_name
order by sum(m.revenue) desc''')

,genre_name,sum(m.revenue)
0,Adventure,25124972342
1,Action,21036581432
2,Science Fiction,18279642305
3,Fantasy,8807960163
4,Family,6767339944
5,Animation,3846875424
6,Thriller,3186760879
7,Romance,3108555314
8,Drama,1845034188


__Get all Genres for the movie "Frozen II".__

In [36]:
my_sql('''select g.genre_name
from genres g join movies m on m.id=g.id
where m.title ="Frozen II"''')

,genre_name
0,Adventure
1,Animation
2,Family


In [37]:
con.close

<function Connection.close>